# Explanation:
Given SHOCK get to VALUE by only changing one letter at a time.

This problem can be formatted as a shortest path problem in a graph, where each word is a node and nodes are connected by an edge if their edit distance is exactly 1.

In [1]:
import itertools
from collections import defaultdict
import requests

In [2]:
page = requests.get("http://www.bestwordlist.com/4letterwords.txt").text.strip().split("\n")
page.pop(0).strip()

'ï»¿There are 5454 four-letter words, http://www.bestwordlist.com/4letterwords.htm'

In [3]:
words = []
for line in page:
    words += line.strip().split()
words = tuple(w.lower() for w in words)

In [4]:
import scipy.misc
print("words:", len(words))
print("worst case: {} edges".format(int(scipy.misc.comb(len(words), 2))))                                                        

words: 5454
worst case: 14870331 edges


In [5]:
word_network = defaultdict(set)

counter = 0
counter_2 = 0
for w1, w2 in itertools.product(range(len(words)), repeat=2):
    counter +=1
    
    if w1 in word_network[w2]:
        continue
    
    if counter % 3000000 == 0:
        print(counter)
    
    c = 0
    for c1, c2 in zip(words[w1], words[w2]):
        if c1 != c2:
            c += 1
            
    if 1 == c:
        word_network[w1].add(w2)
        counter_2 += 1

len(word_network), "with ", counter_2

3000000
6000000
9000000
12000000
15000000
18000000
21000000
24000000
27000000


(5454, 'with ', 35905)

## Some Kind of Statistics over Network

### Average Connectivity

In [6]:
s = 0
for key in word_network:
    s += len(word_network[key])
int(s / len(word_network))

6

# Shortest Path with Neo4J

In [7]:
from py2neo import Graph
graph = Graph()

Before begginning, delete all Word and Hop entries from graph

In [8]:
graph.cypher.execute("MATCH (n:Word) DETACH DELETE n")
graph.cypher.execute("MATCH (u:Word)-[r:HOP]-(v:WORD) DETACH DELETE r")

Add all Words to graph and index them

In [9]:
with graph.cypher.begin() as cy:
    for word in words:
        cy.append("CREATE (:Word {value: {word}})", word=word)

graph.cypher.execute("CREATE INDEX ON :Word(value)")

Add all relationships to graph

In [10]:
q = "MATCH (u:Word {value:'«r1»'}), (v:Word {value:'«r2»'}) CREATE (u)-[c:HOP]->(v)"
c = 0
for i in range(len(words)):
    for j in word_network[i]:
        c += 1
        if c % 5000 == 0:
            print(c)
        graph.cypher.execute(q, r1=words[i], r2=words[j])

5000
10000
15000
20000
25000
30000
35000


Compute shortest path(s)

In [11]:
graph.cypher.execute("""MATCH (u:Word {value:"cake"}), (v:Word {value:"pint"}), p = shortestPath((u)-[*..10]-(v)) RETURN p""")

   | p                                                                                                                                                 
---+----------------------------------------------------------------------------------------------------------------------------------------------------
 1 | (:Word {value:"cake"})-[:HOP]->(:Word {value:"cane"})-[:HOP]->(:Word {value:"pane"})-[:HOP]->(:Word {value:"pine"})-[:HOP]->(:Word {value:"pint"})